In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder,LabelEncoder
import pickle

In [3]:
df=pd.read_csv("/Users/rahulmanchanda/Desktop/deep-learning-ann/Churn_Modelling.csv")
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
df.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1, inplace=True)

In [5]:
label_encoder_gender=LabelEncoder()
df["Gender"]=label_encoder_gender.fit_transform(df["Gender"])

In [6]:
onehot_encoder_geography=OneHotEncoder()
geoencoder=onehot_encoder_geography.fit_transform(df[["Geography"]])
geo_encoded_df=pd.DataFrame(geoencoder.toarray(), columns=onehot_encoder_geography.get_feature_names_out(["Geography"]))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [7]:
data=pd.concat([df.drop("Geography",axis=1), geo_encoded_df], axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [9]:
X=data.drop("EstimatedSalary", axis=1)
y=data["EstimatedSalary"]

In [11]:
y

0       101348.88
1       112542.58
2       113931.57
3        93826.63
4        79084.10
          ...    
9995     96270.64
9996    101699.77
9997     42085.58
9998     92888.52
9999     38190.78
Name: EstimatedSalary, Length: 10000, dtype: float64

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [13]:
with open("label_encoder_gender.pkl", "wb") as f:
    pickle.dump(label_encoder_gender,f)
with open("onehot_encoder_geography.pkl", "wb") as f:
    pickle.dump(onehot_encoder_geography,f) 
with open("scaler.pkl", "wb") as f:
    pickle.dump(scaler,f)

In [14]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


In [15]:
model=Sequential(
    [
        Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
        Dense(32, activation='relu'),
        Dense(1)  # No activation function for regression output
    ]
)
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [18]:
from tensorflow.keras.callbacks import TensorBoard
import datetime
log_dir = "regressionlogs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [19]:
early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5,restore_best_weights=True)

In [20]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=100,
    batch_size=32,
    callbacks=[tensorboard_callback, early_stopping_callback]
)

Epoch 1/100
250/250 [==============================] - 0s 693us/step - loss: 13384171520.0000 - mae: 100380.0234 - val_loss: 13009940480.0000 - val_mae: 98533.0156
Epoch 2/100
250/250 [==============================] - 0s 483us/step - loss: 13248710656.0000 - mae: 99704.0547 - val_loss: 12740194304.0000 - val_mae: 97169.8438
Epoch 3/100
250/250 [==============================] - 0s 391us/step - loss: 12771900416.0000 - mae: 97340.7578 - val_loss: 12051402752.0000 - val_mae: 93691.5703
Epoch 4/100
250/250 [==============================] - 0s 400us/step - loss: 11824896000.0000 - mae: 92660.4922 - val_loss: 10887399424.0000 - val_mae: 87813.5547
Epoch 5/100
250/250 [==============================] - 0s 392us/step - loss: 10431127552.0000 - mae: 85692.8125 - val_loss: 9350726656.0000 - val_mae: 80055.3672
Epoch 6/100
250/250 [==============================] - 0s 392us/step - loss: 8759996416.0000 - mae: 77357.1562 - val_loss: 7670763520.0000 - val_mae: 71656.6406
Epoch 7/100
250/250 [===

In [22]:
%load_ext tensorboard


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [27]:
%tensorboard --logdir regressionlogs/fit/

Reusing TensorBoard on port 6008 (pid 32150), started 0:00:55 ago. (Use '!kill 32150' to kill it.)

In [28]:
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f"Test MAE: {test_mae}")      
print(f"Test Loss: {test_loss}")

63/63 [==============================] - 0s 232us/step - loss: 3349175296.0000 - mae: 50060.9297
Test MAE: 50060.9296875
Test Loss: 3349175296.0


In [29]:
model.save("salary_regression_model.h5")

/Users/rahulmanchanda/Desktop/deep-learning-ann/venv311/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
